# Hvad er det her?
____
### Jeg tænkte det kunne være interessant at kigge lidt på folketingets stemme data.
### Det her dokument går lidt løst frem og undersøger diverse aspekter om data'ene undervejs.
### Det betyder at del-resultaterne ikke bør tages for 100% sandhed da jeg ikke har det fulde billede endnu

# Lidt info at huske på

## 4 stemmetyper:
    * 1: For
    * 2: Imod
    * 3: Fraværende
    * 4: Hverken for eller imod
    
    
## 12 Aktørtyper
    * 4: Parti
    * 5: Person

# Hvad kan vi så spørge om...
* Hvem stemmer mest nej? (person, parti)
* Hvem stemmer mest ja?  (person, parti)
* Hvis vi køre PCA (eller MCA eller CorEx) på personer, kommer de så til at cluster efter parti? Er der nogen der hører til et andet sted?
* Hvis vi gør det samme på parti niveau ser vi så vestre vs højre fløj?
* Hvem er den mest/mindst parti loyale person?
* ...
* Er der et mindre antal afstemninger der hvis samlede resultat kan adskille alle partierne således at de kunne bruges som en test til at se hvem du er mest enige med?

In [1]:
#import ting og sager til grafer og databehandling
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

#import django til at læse databasen
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'politik.settings')
import django
django.setup()
from django.db.models import Count, Avg

from scipy.stats import linregress

from poligraf import models
from datetime import datetime
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import HoverTool, Range1d
from bokeh.mpl import to_bokeh
from bokeh.charts import Bar
output_notebook()

Loading BokehJS ...

In [2]:
#Der er en del partier der har skiftet navn gennem årerne så her er en dict til at holde styr på det
parti_dual_navn = {
    'Alternativet':'Alternativet',
    'Centrum-demokraterne':'Centrum-demokraterne',
    'Danmarks Retsforbund':'Danmarks Retsforbund',
    'Danmarks kommunistiske parti':'Danmarks kommunistiske parti',
    'Dansk Folkeparti':'Dansk Folkeparti',
    'De frie demokrater':'De frie demokrater',
    'De moderate':'De moderate',
    'De uafhængige':'De uafhængige',
    'Demokratisk Folkeparti':'Demokratisk Folkeparti',
    'Den socialistiske arbejdsgruppe':'Den socialistiske arbejdsgruppe',
    'Det Konservative Folkeparti':'Det Konservative Folkeparti',
    'Det Liberale Højre':'Det Liberale Højre',
    'Det Radikale Venstre':'Radikale Venstre',
    'Enhedslisten':'Enhedslisten',
    'Enhedslisten - De Rød-Grønne':'Enhedslisten',
    'Erhvervspartiet':'Erhvervspartiet',
    'Folkeunionen':'Folkeunionen',
    'Fredspolitisk folkeparti':'Fredspolitisk folkeparti',
    'Fremskridtspartiet':'Fremskridtspartiet',
    'Frihed 2000':'Fremskridtspartiet',
    'Fælles Kurs':'Fælles Kurs',
    'Fællespartiet':'Fællespartiet',
    'Fólkaflokkurin':'Fólkaflokkurin',
    'Inuit Ataqatigiit':'Inuit Ataqatigiit',
    'Javnaðarflokkurin':'Javnaðarflokkurin',
    'Kristeligt Folkeparti':'Kristeligt Folkeparti',
    'Kristeligt folkeparti':'Kristeligt Folkeparti',
    'Kristendemokraterne':'Kristendemokraterne',
    'Liberal Alliance':'Liberal Alliance',
    'Liberalt centrum':'Liberalt centrum',
    'Ny Alliance':'Liberal Alliance',
    'Radikale Venstre':'Radikale Venstre',
    'Sambandsflokkurin':'Sambandsflokkurin',
    'Siumut':'Siumut',
    'Slesvigsk parti':'Slesvigsk parti',
    'Socialdemokratiet':'Socialdemokratiet',
    'Socialistisk Folkeparti':'Socialistisk Folkeparti',
    'Tjóðveldi':'Tjóðveldi',
    'Tjóðveldisflokkurin':'Tjóðveldisflokkurin',
    'Trivselspartiet':'Trivselspartiet',
    'Uden for folketingsgrupperne - Anders Møller':'Uden for folketingsgrupperne - Anders Møller',
    'Uden for folketingsgrupperne - Anders Storgaard':'Uden for folketingsgrupperne - Anders Storgaard',
    'Uden for folketingsgrupperne - Anker Tang Sørensen':'Uden for folketingsgrupperne - Anker Tang Sørensen',
    'Uden for folketingsgrupperne - Anne Grete Holmsgaard':'Uden for folketingsgrupperne - Anne Grete Holmsgaard',
    'Uden for folketingsgrupperne - Arne Bjerregaard':'Uden for folketingsgrupperne - Arne Bjerregaard',
    'Uden for folketingsgrupperne - Arne Melchior':'Uden for folketingsgrupperne - Arne Melchior',
    'Uden for folketingsgrupperne - Asger Lindinger':'Uden for folketingsgrupperne - Asger Lindinger',
    'Uden for folketingsgrupperne - Bente Juncker':'Uden for folketingsgrupperne - Bente Juncker',
    'Uden for folketingsgrupperne - Bjørn Elmquist':'Uden for folketingsgrupperne - Bjørn Elmquist',
    'Uden for folketingsgrupperne - Børge Diderichsen':'Uden for folketingsgrupperne - Børge Diderichsen',
    'Uden for folketingsgrupperne - Christian H. Hansen':'Uden for folketingsgrupperne - Christian H. Hansen',
    'Uden for folketingsgrupperne - Erik A. Jensen':'Uden for folketingsgrupperne - Erik A. Jensen',
    'Uden for folketingsgrupperne - Erling Askjær Jørgensen':'Uden for folketingsgrupperne - Erling Askjær Jørgensen',
    'Uden for folketingsgrupperne - Flemming Oppfeldt':'Uden for folketingsgrupperne - Flemming Oppfeldt',
    'Uden for folketingsgrupperne - Frank Dahlgaard':'Uden for folketingsgrupperne - Frank Dahlgaard',
    'Uden for folketingsgrupperne - Gitte Seeberg':'Uden for folketingsgrupperne - Gitte Seeberg',
    'Uden for folketingsgrupperne - Hans Jørgen Iversen':'Uden for folketingsgrupperne - Hans Jørgen Iversen',
    'Uden for folketingsgrupperne - Hans Jørgen Lembourn':'Uden for folketingsgrupperne - Hans Jørgen Lembourn',
    'Uden for folketingsgrupperne - Herluf Rasmussen':'Uden for folketingsgrupperne - Herluf Rasmussen',
    'Uden for folketingsgrupperne - Hugo Holm':'Uden for folketingsgrupperne - Hugo Holm',
    'Uden for folketingsgrupperne - Inge Refshauge':'Uden for folketingsgrupperne - Inge Refshauge',
    'Uden for folketingsgrupperne - Jacob Haugaard ':'Uden for folketingsgrupperne - Jacob Haugaard ',
    'Uden for folketingsgrupperne - Jørgen Junior':'Uden for folketingsgrupperne - Jørgen Junior',
    'Uden for folketingsgrupperne - Jørgen Lenger':'Uden for folketingsgrupperne - Jørgen Lenger',
    'Uden for folketingsgrupperne - Jørgen Poulsen':'Uden for folketingsgrupperne - Jørgen Poulsen',
    'Uden for folketingsgrupperne - Kim Behnke':'Uden for folketingsgrupperne - Kim Behnke',
    'Uden for folketingsgrupperne - Kirsten Jacobsen':'Uden for folketingsgrupperne - Kirsten Jacobsen',
    'Uden for folketingsgrupperne - Leif Mikkelsen':'Uden for folketingsgrupperne - Leif Mikkelsen',
    'Uden for folketingsgrupperne - Louise Frevert':'Uden for folketingsgrupperne - Louise Frevert',
    'Uden for folketingsgrupperne - Mogens Andreasen':'Uden for folketingsgrupperne - Mogens Andreasen',
    'Uden for folketingsgrupperne - Morten Messerschmidt':'Uden for folketingsgrupperne - Morten Messerschmidt',
    'Uden for folketingsgrupperne - Naser Khader':'Uden for folketingsgrupperne - Naser Khader',
    'Uden for folketingsgrupperne - Niels Vestergaard Eriksen':'Uden for folketingsgrupperne - Niels Vestergaard Eriksen',
    'Uden for folketingsgrupperne - Niels Westerby':'Uden for folketingsgrupperne - Niels Westerby',
    'Uden for folketingsgrupperne - Ole Donner':'Uden for folketingsgrupperne - Ole Donner',
    'Uden for folketingsgrupperne - Per Ørum Jørgensen':'Uden for folketingsgrupperne - Per Ørum Jørgensen',
    'Uden for folketingsgrupperne - Peter Brixtofte':'Uden for folketingsgrupperne - Peter Brixtofte',
    'Uden for folketingsgrupperne - Pia Christmas-Møller':'Uden for folketingsgrupperne - Pia Christmas-Møller',
    'Uden for folketingsgrupperne - Simon Emil Ammitzbøll':'Uden for folketingsgrupperne - Simon Emil Ammitzbøll',
    'Uden for folketingsgrupperne - Svend Bay':'Uden for folketingsgrupperne - Svend Bay',
    'Uden for folketingsgrupperne - Thorkild B. Fransgaard':'Uden for folketingsgrupperne - Thorkild B. Fransgaard',
    'Uden for folketingsgrupperne - Thure Barsøe-Carnfeldt':'Uden for folketingsgrupperne - Thure Barsøe-Carnfeldt',
    'Uden for folketingsgrupperne - Uffe Elbæk':'Uden for folketingsgrupperne - Uffe Elbæk',
    'Uden for folketingsgrupperne - Uffe Thorndal':'Uden for folketingsgrupperne - Uffe Thorndal',
    'Uden for folketingsgrupperne - valgt i Grønland':'Uden for folketingsgrupperne - valgt i Grønland',
    'Uden for folketingsgrupperne - valgt på Færøerne':'Uden for folketingsgrupperne - valgt på Færøerne',
    'Venstre, Danmarks Liberale Parti':'Venstre, Danmarks Liberale Parti',
    'Venstresocialisterne':'Venstresocialisterne'
}

In [14]:
#Vi laver data'ene om til en dataframe med afstemning, dato, navn og hvad folk stemte

stemmetyper = {1:'for', 2:'imod', 3:'fraværende', 4:'neutral'}
stemmer = pd.DataFrame([{'afstemning':stemme.afstemningid.pk, 
            'stemt': stemme.typeid_id,#stemmetyper[stemme.typeid_id], 
            'navn': stemme.aktørid.navn,
            'navn_id': stemme.aktørid_id,
            'dato':datetime.strptime(stemme.afstemningid.mødeid.dato.split("T")[0], "%Y-%m-%d")} 
           for stemme in models.Stemme.objects.all().prefetch_related('aktørid', 'afstemningid__mødeid')])
stemmer.head()

,afstemning,dato,navn,navn_id,stemt
0,1,2014-09-09,Eigil Andersen,158,1
1,1,2014-09-09,Tom Behnke,71,1
2,1,2014-09-09,Liselott Blixt,50,1
3,1,2014-09-09,Erling Bonnesen,49,1
4,1,2014-09-09,Bent Bøgsted,183,1


In [15]:
#Men ofte vil det også være interessant hvilket parti folk er medlem af
#Da folk kan skifte parti, så kommer det an på hvornår afstemningen var.
#Alt i denne celle laver en oversigt over hvornår hvem var i hvilket parti
parti_assigner = pd.DataFrame([pp for pp in models.Aktør.objects.filter(typeid_id=5, 
                                                       tilaktør__fraaktørid__typeid_id=4).values_list(
            'pk', 'tilaktør__fraaktørid__navn', 'tilaktør__startdato', 'tilaktør__slutdato')] + 
             [pp for pp in models.Aktør.objects.filter(typeid_id=5,
                                                       fraaktør__tilaktørid__typeid_id=4).values_list(
            'pk', 'fraaktør__tilaktørid__navn', 'fraaktør__startdato', 'fraaktør__slutdato')], columns=['id', 'parti', 'fra', 'til'])

def unify_navn(parti):
    return parti_dual_navn[parti]

def make_dato(dato):
    if dato != None:
        return datetime.strptime(dato.split("T")[0], "%Y-%m-%d")
    return np.nan

parti_assigner['parti'] = parti_assigner['parti'].apply(unify_navn)
parti_assigner['fra'] = parti_assigner['fra'].apply(make_dato)
parti_assigner['til'] = parti_assigner['til'].apply(make_dato)
parti_assigner.drop_duplicates(inplace=True)
parti_assigner.head()

def simplify(pa):
    new_data_rows = []
    for id in pa.id.unique():
        pas = pa[pa['id'] == id]['parti'].unique()
        if len(pas) == 1:
            new_data_rows.append({'id':id,'parti':pas[0], 'fra':datetime(1900, 1, 1), 'til':datetime(2100, 1, 1)})
        else:
            new_data_rows += pa[pa['id']==id].to_dict('records')
    return new_data_rows
            
parti_assigner = pd.DataFrame(simplify(parti_assigner))
parti_assigner.head()

,fra,id,parti,til
0,1900-01-01,951,Socialdemokratiet,2100-01-01
1,1900-01-01,1727,Socialdemokratiet,2100-01-01
2,1900-01-01,386,Socialdemokratiet,2100-01-01
3,1900-01-01,235,Socialdemokratiet,2100-01-01
4,1900-01-01,652,Socialdemokratiet,2100-01-01


In [16]:
#den info kan vi så bruge på at placere folk i deres respektive partier på de givne afstemningstidpunkter

def party_time(row):
    #brug afstemningsdato og person id til at afgøre medlemskab på dato
    a = parti_assigner[(parti_assigner.id == row['navn_id']) & (parti_assigner.fra <= row['dato'])]
    if len(a)==0:
        print(row)
        if row['navn_id'] == 16074: #Steen Holm Iversen er ikke skrevet korrekt ind...han er LA
            return "Liberal Alliance"
        return "error"
    else:
        return a.sort_values(by='fra', ascending=False).iloc[0]['parti']
stemmer['parti'] = stemmer.apply(party_time, axis=1)

afstemning                   2645
dato          2015-10-22 00:00:00
navn           Steen Holm Iversen
navn_id                     16074
stemt                           3
Name: 139977, dtype: object
afstemning                   2646
dato          2015-10-22 00:00:00
navn           Steen Holm Iversen
navn_id                     16074
stemt                           3
Name: 140156, dtype: object
afstemning                   2648
dato          2015-11-03 00:00:00
navn           Steen Holm Iversen
navn_id                     16074
stemt                           2
Name: 140260, dtype: object
afstemning                   2649
dato          2015-11-03 00:00:00
navn           Steen Holm Iversen
navn_id                     16074
stemt                           1
Name: 140437, dtype: object


In [17]:
stemmer.head()

,afstemning,dato,navn,navn_id,stemt,parti
0,1,2014-09-09,Eigil Andersen,158,1,Socialistisk Folkeparti
1,1,2014-09-09,Tom Behnke,71,1,Det Konservative Folkeparti
2,1,2014-09-09,Liselott Blixt,50,1,Dansk Folkeparti
3,1,2014-09-09,Erling Bonnesen,49,1,"Venstre, Danmarks Liberale Parti"
4,1,2014-09-09,Bent Bøgsted,183,1,Dansk Folkeparti


In [7]:
#nå, så nogle spørgsmål: hvilket parti er mindst intern enig?

parti_basis = {}

for af in stemmer.afstemning.unique():
    for parti in stemmer[stemmer.afstemning == af]['parti'].unique():
        if parti not in parti_basis:
            parti_basis[parti] = [0,0]
        parti_basis[parti][1] += 1
    b1 = stemmer[(stemmer.afstemning == af) & (stemmer.stemt == 1)]['parti'].unique()
    b2 = stemmer[(stemmer.afstemning == af) & (stemmer.stemt == 2)]['parti'].unique()
    both_sides = set(b1).intersection(b2)
    if len(both_sides) != 0:
        for parti in both_sides:
            parti_basis[parti][0] += 1

In [8]:
bla = pd.DataFrame(parti_basis).T
blabla = (bla[0]/bla[1])*100
print(blabla[blabla>0])
show(Bar(blabla[blabla>0], title="følger folk partiet?", ylabel="% afstemninger >1 imod flertallet"))

Dansk Folkeparti                    0.462963
Det Konservative Folkeparti         0.370370
Enhedslisten                        0.277778
Liberal Alliance                    0.833333
Radikale Venstre                    0.277778
Socialdemokratiet                   0.925926
Socialistisk Folkeparti             2.500000
Venstre, Danmarks Liberale Parti    0.833333
dtype: float64


## Ok så der er af og til nogen der stemmer imod partiet ...er der nogen specifikt der gør det eller er det bare lige en enkelt person fra tid til anden?

In [9]:
person_basis = {}

for af in stemmer.afstemning.unique():
    for person in stemmer[stemmer.afstemning == af]['navn'].unique():
        if person not in person_basis:
            person_basis[person] = [0,0]
        person_basis[person][1] += 1
    b1 = stemmer[(stemmer.afstemning == af) & (stemmer.stemt == 'for')]['parti'].unique()
    b2 = stemmer[(stemmer.afstemning == af) & (stemmer.stemt == 'imod')]['parti'].unique()
    both_sides = set(b1).intersection(b2)
    if len(both_sides) != 0:
        for parti in both_sides:
            whoisit = stemmer[(stemmer.afstemning == af) & (stemmer.parti == parti)]
            if len(whoisit[whoisit.stemt == 'for']) > len(whoisit[whoisit.stemt == 'imod']):
                for person in whoisit[whoisit.stemt == 'imod']['navn']:
                    person_basis[person][0] +=1
            else:
                for person in whoisit[whoisit.stemt == 'for']['navn']:
                    person_basis[person][0] +=1

In [10]:
bla = pd.DataFrame(person_basis).T
blabla = (bla[0]/bla[1])*100
show(Bar(blabla[blabla>0], title="følger folk partiet?", ylabel="% stemmer imod parti flertallet"))

## Hm hm...en mindre håndfuld få gange....hovedsageligt bliver partilinjen fulgt

In [11]:
sum_stem = stemmer.pivot(index='afstemning', columns='navn', values='stemt').apply(pd.value_counts).T
sum_stem.head()

p = figure()
p.circle('for', 'imod', radius=5, source=ColumnDataSource(sum_stem))
p.xaxis.axis_label = "for"
p.yaxis.axis_label = 'imod'
p.add_tools(HoverTool(tooltips=[('name', "@navn")]))

#slope, intercept, r_value, p_value, std_err = linregress(stemt_data.dropna()['imod'], stemt_data.dropna()['for'])

#p.line(range(500), [slope*x+intercept for x in range(500)])
#p.text([400, 400, 400], [400, 420, 440], [
#        'Intercept: {:.2f}'.format(intercept), 
#        'Slope: {:.2f}'.format(slope),
#        'r^2: {:.2f}'.format(r_value**2)])

#p.set(x_range=Range1d(0, 600), y_range=Range1d(0,600))
show(p)

In [533]:
# så per afstemning skal vi skifte personer ud med parti-linjen
# det bliver en væsentligt mindre dataframe.
# den kan vi så sætte op som en parti x afstemning array med for/imod per afstemning
# det burde kunne kastes ind i noget PCA...=)

,afstemning,dato,navn,navn_id,stemt,parti
0,1,2014-09-09,Eigil Andersen,158,for,Socialistisk Folkeparti
1,1,2014-09-09,Tom Behnke,71,for,Det Konservative Folkeparti
2,1,2014-09-09,Liselott Blixt,50,for,Dansk Folkeparti
3,1,2014-09-09,Erling Bonnesen,49,for,"Venstre, Danmarks Liberale Parti"
4,1,2014-09-09,Bent Bøgsted,183,for,Dansk Folkeparti


In [36]:
parti_stemme = []

for af in stemmer.afstemning.unique():
    b1 = stemmer[(stemmer.afstemning == af) & (stemmer.stemt == 1)]['parti']
    b2 = stemmer[(stemmer.afstemning == af) & (stemmer.stemt == 2)]['parti']
    c1 = set(b1)
    c2 = set(b2)
    both_sides = c1.intersection(c2)
    if len(both_sides) != 0:
        for parti in both_sides:
            if sum(b1==parti)>sum(b2==parti):
                c2.remove(parti)
            else:
                c1.remove(parti)
    parti_stemme.append({**{key:1 for key in c1},**{key:2 for key in c2}})

In [45]:
parti_stemme_df = pd.DataFrame(parti_stemme)
parti_stemme_df = parti_stemme_df.fillna(0)

In [75]:
full_time = parti_stemme_df[['Dansk Folkeparti', 'Det Konservative Folkeparti', 'Enhedslisten', 'Liberal Alliance', 'Radikale Venstre', 'Socialdemokratiet', 'Socialistisk Folkeparti', 'Venstre, Danmarks Liberale Parti']]


In [22]:
from sklearn.decomposition import PCA

In [79]:
pca = PCA(n_components=1)

In [80]:
pcad = pca.fit_transform(full_time.T)

In [129]:
pca_med_navn = pd.DataFrame(data=pcad, index=full_time.T.index).reset_index()
pca_med_navn.columns = ['parti', 'pca1']
pca_med_navn['statisk'] = [0 for x in range(len(full_time.T.index))]
pca_med_navn['pca1'] *= -1
pca_med_navn = pca_med_navn.sort_values('pca1')

In [134]:
p = figure(width=750, height=400)
p.circle('pca1', 'statisk', radius=0.1, alpha=0.5, source=ColumnDataSource(pd.DataFrame(pca_med_navn)))
p.text(pca_med_navn['pca1'], 0, pca_med_navn['parti'], angle=[0.5,-0.5] * int(len(pca_med_navn)/2))
p.xaxis.axis_label = "pca1"
p.yaxis.axis_label = ''
p.add_tools(HoverTool(tooltips=[('parti', "@parti")]))

show(p)

In [89]:
plt.hlines(1,0,30)  # Draw a horizontal line
plt.eventplot(pca_med_navn['pca1']+10, orientation='horizontal', colors='b')
plt.show()

In [122]:
[-1,1] * 3

[-1, 1, -1, 1, -1, 1]